In [1]:
from dotenv import load_dotenv
import os
import requests
import json
import datetime
import sys
import time

load_dotenv()

from IPython.display import Image, display

In [2]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool, tool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

In [3]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = AzureChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        if not isinstance(input, str):
            raise ValueError("Input must be a string")
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")
        return result

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool._run),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [4]:
@tool
def calculate_text_length(text: str) -> int:
    """
    Calculate the length of the given text.

    This function is useful for determining the number of characters in a string,
    which is necessary to enforce character limits.

    Args:
        text (str): The input text to measure.

    Returns:
        int: The length of the input text.
    """
    return len(text)

In [5]:
tools = [calculate_text_length]
test_agent = MyAgent(tools=tools, custom_prompt="You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID")

In [6]:
result = test_agent.myinvoke("Calculate the length of this text")
print(result)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Please provide the text for which you want to calculate the length.

> Finished chain.
{'input': 'Calculate the length of this text', 'chat_history': [HumanMessage(content='Calculate the length of this text'), AIMessage(content='Please provide the text for which you want to calculate the length.')], 'output': 'Please provide the text for which you want to calculate the length.'}

--> Agent Response:
Please provide the text for which you want to calculate the length.

{'input': 'Calculate the length of this text', 'chat_history': [HumanMessage(content='Calculate the length of this text'), AIMessage(content='Please provide the text for which you want to calculate the length.')], 'output': 'Please provide the text for which you want to calculate the length.'}


In [7]:
while True:
    user_message = input("Enter a message: ")
    if user_message == "exit":
        break
    result = test_agent.myinvoke(user_message)
    print(result)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `calculate_text_length` with `{'text': 'Calculate the length of this text'}`


33The length of the text "Calculate the length of this text" is 33 characters.

> Finished chain.
{'input': 'The text was "Calculate the length of this text"', 'chat_history': [HumanMessage(content='Calculate the length of this text'), AIMessage(content='Please provide the text for which you want to calculate the length.'), HumanMessage(content='The text was "Calculate the length of this text"'), AIMessage(content='The length of the text "Calculate the length of this text" is 33 characters.')], 'output': 'The length of the text "Calculate the length of this text" is 33 characters.'}

--> Agent Response:
The length of the text "Calculate the length of this text" is 33 characters.

{'input': 'The text was "Calculate the length of this text"', 'chat_history': [HumanMessage(content='Calculate the length of this text'), AIMessage(content='Please provide the text for which you want to calculate the leng

In [5]:
def cli_print(text: str):
    print("===================================")
    print(text)
    print("---------------------------------------\n")

In [ ]:
story = {
    "title": "Livinha e a Chave Encantada: Em busca das luzes roubadas",
    "scenes": [
        {
            "number": 0,
            "title": "Book Cover for 'Livinha e o Resgate da Luz Celestial'",
            "content": "Lívia holds The Key in her hand, standing before The Magic Door. The forest around her is filled with glowing trees and magical creatures. The sky above is dark, but the stars and moon are missing. A golden hawk watches from a branch, its eyes full of wisdom. The cover reads: 'Livinha e o Resgate da Luz Celestial'.",
            "characters": ["Lívia"],
            "items": ["The Key", "The Magic Door"]
        },
        {
            "number": 1,
            "title": "Cena 1: Livinha encontra a chave dourada",
            "content": "As últimas noites estavam estranhamente escuras, sem lua ou estrelas, mesmo quando deveria haver uma lua cheia iluminando o céu. Esse mistério deixava todos inquietos, mas naquela manhã ensolarada, Livinha decidiu esquecer o medo e explorar a floresta. Entre as folhas caídas, algo brilhou. Ela se abaixou e encontrou uma chave dourada, com desenhos de estrelas, luas e galhos gravados. “O que será que ela abre?” pensou, com os olhos brilhando de empolgação. O leve brilho da chave parecia chamá-la, e Livinha, sem hesitar, decidiu seguir o caminho misterioso que se abria à sua frente.",
            "characters": ["Lívia"],
            "items": ["The Key"]
        },
        {
            "number": 2,
            "title": "Cena 2: A porta mágica na árvore gigante",
            "content": "Enquanto caminhava pela floresta, Livinha parou ao avistar algo extraordinário. Suspensa no ar, diante de uma clareira banhada de luz suave, estava uma pequena porta arredondada, feita de madeira escura e lisa. No centro, uma estrela dourada brilhava discretamente. Livinha segurou a chave dourada com firmeza. “É aqui,” sussurrou para si mesma, o coração acelerado de expectativa. Com cuidado, ela encaixou a chave na fechadura. Assim que girou, a estrela dourada brilhou intensamente, envolvendo o portal quando ele se abriu, revelando um mundo mágico além. Livinha hesitou apenas por um instante. Ao atravessar, a porta se fechou atrás dela e desapareceu no ar, deixando apenas o sussurro do vento na clareira silenciosa.",
            "characters": ["Lívia"],
            "items": ["The Key", "The Magic Door"]
        },
        {
            "number": 3,
            "title": "Cena 3: A floresta das árvores brilhantes",
            "content": "Do outro lado, Livinha ficou encantada. A floresta era diferente de tudo que ela já tinha visto, com árvores cujas folhas brilhavam suavemente como se fossem feitas de luz. O ar tinha um perfume doce, e vaga-lumes dançavam ao seu redor. Contudo, algo parecia errado. O céu, que deveria estar cheio de estrelas, estava vazio e escuro, como se tivesse perdido seu brilho. Logo, um falcão de penas douradas pousou em um galho baixo e a observou com olhos curiosos. “Quem é você e como chegou até aqui?” perguntou o falcão com uma voz calma, mas séria. Livinha explicou sobre a chave que encontrara, e os animais trocaram olhares preocupados.",
            "characters": ["Lívia", "Corina", "Forest Animals"],
            "items": ["The Key"]
        },
        {
            "number": 4,
            "title": "Cena 4: Corina, o falcão sábio, explica o problema",
            "content": "“Eu sou Corina, a última Sentinela da floresta mágica,” disse o falcão. “E você chegou em um momento muito difícil. A luz das estrelas e da lua foi roubada, e a floresta está enfraquecendo.” Livinha arregalou os olhos, surpresa. “Quem faria isso?” perguntou. Corina suspirou. “Há muitos anos, uma vilã chamada Tenebris lançou uma maldição sobre minha família, nos separando e enfraquecendo nossa proteção. Quando meu pai partiu, fiquei sozinha, e a floresta perdeu sua força. Foi então que a luz foi roubada.” Corina mostrou a Livinha uma lanterna antiga, com um símbolo dos Sentinelas gravado. “Esta lanterna pertencia ao meu pai, mas apagou quando ele se foi. Agora, ela brilhou novamente e escolheu você. Foi a lanterna que enviou a chave para guiá-la até aqui. Ela acredita que você é quem pode trazer a luz de volta.”",
            "characters": ["Lívia", "Corina"],
            "items": ["The Lantern", "The Key"]
        },
        {
            "number": 5,
            "title": "Cena 5: Recebendo o mapa e conhecendo Marildo",
            "content": "Enquanto Corina falava, um som de galhos quebrando chamou a atenção de Livinha. “Quem está aí?” perguntou Corina, franzindo as penas. Um coelho branco com um moletom azul claro saiu hesitante de trás de um arbusto. “Eu só estava ouvindo! Não pude evitar!” disse ele, envergonhado. “Eu sou o Coelho Marildo!” continuou ele, tentando parecer corajoso. “E conheço bem os caminhos da floresta. Deixe-me ajudar!” Livinha riu com sua animação e aceitou sua ajuda. Corina entregou a lanterna a Livinha e um mapa mágico, avisando: “O caminho será perigoso, mas a lanterna iluminará seu caminho e protegerá você.”",
            "characters": ["Lívia", "Corina", "Marildo"],
            "items": ["The Lantern", "The Map"]
        },
        {
            "number": 6,
            "title": "Cena 6: O riacho cintilante e o enigma da luz antiga",
            "content": "Depois de horas caminhando, Livinha e Marildo chegaram a um riacho cujas águas brilhavam como estrelas líquidas. “O mapa diz que precisamos atravessar aqui,” disse Livinha. Mas não havia ponte ou passagem visível. Uma pedra próxima começou a brilhar com palavras mágicas: \"Somente uma luz antiga, do coração da floresta, pode guiar o caminho.\" Livinha tentou apontar a lanterna para o riacho, mas nada aconteceu. Marildo, com olhos atentos, percebeu um padrão na água. “Tente iluminar aquele ponto ali,” disse ele, apontando para uma pedra com marcas semelhantes às da chave. Quando Livinha fez isso, a lanterna criou um caminho de luz flutuante. “Eu sabia que era isso!” exclamou Marildo, orgulhoso. Juntos, cruzaram o riacho, as águas parecendo sussurrar em aprovação.",
            "characters": ["Lívia", "Marildo"],
            "items": ["The Lantern", "The Map"]
        },
        {
            "number": 7,
            "title": "Cena 7: O covil de Tenebris",
            "content": "Mais adiante, Livinha e Marildo entraram em uma enorme câmara subterrânea, onde a luz das estrelas e da lua estava presa em um grande orbe flutuante. Uma figura imponente estava de pé diante de um altar. Era Tenebris. Ao lado dela, acorrentado e de joelhos, estava uma criatura envolta em sombras, com olhos cheios de dor. “Chegaram tarde demais,” disse Tenebris, sua voz ecoando pelo espaço. “O ritual está quase completo. Com este poder, destruirei os Sentinelas e sua magia para sempre.” Livinha sentiu o peso das palavras, mas olhou para a criatura e viu algo em seus olhos—uma súplica silenciosa. “Pare com isso! Você não precisa fazer isso,” disse Livinha, aproximando-se. “Se precisar de alguém para o ritual, leve a mim.” Marildo arregalou os olhos. “O quê? Não!” sussurrou ele, alarmado. Mas Livinha ergueu o queixo. “Se for para salvar os outros, eu aceito.”",
            "characters": ["Lívia", "Marildo", "Tenebris", "Caelus (Before the Rescue)"],
            "items": ["The Orb", "The Lantern"]
        },
        {
            "number": 8,
            "title": "Cena 8: A fuga e o desmoronamento",
            "content": "Enquanto Tenebris hesitava, surpresa pela oferta de Livinha, Marildo viu sua chance. Silenciosamente, ele contornou a câmara, pegou o orbe que continha as luzes do céu e correu. “Não!” gritou Tenebris, estendendo a mão, mas nesse momento o altar começou a rachar. As paredes da câmara começaram a tremer, e pedaços do teto começaram a cair. Livinha correu até a criatura, que ainda estava acorrentada. Com a lanterna, ela quebrou as correntes que o prendiam. “Vamos sair daqui!” gritou ela. A criatura, livre, segurou Livinha e a ajudou a escapar, mesmo enquanto a câmara desmoronava ao seu redor. Atrás deles, Tenebris ficou encurralada, mas a criatura olhou para trás. “Não posso deixá-la para trás,” disse ele, correndo para salvar sua captora.",
            "characters": ["Lívia", "Marildo", "Tenebris", "Caelus (Before the Rescue)"],
            "items": ["The Orb", "The Lantern"]
        },
        {
            "number": 9,
            "title": "Cena 9: A revelação de Caelus no vilarejo",
            "content": "Quando Livinha, Marildo e a criatura chegaram ao vilarejo, Corina estava esperando ansiosa. Ao ver a criatura, seus olhos dourados se arregalaram. “Essas marcas... não pode ser,” murmurou. Ela desceu rapidamente e parou diante dele, observando as penas escuras marcadas com traços dourados. “Você é meu irmão... Caelus!” exclamou Corina, a voz embargada pela emoção. A criatura ficou imóvel, como se as palavras dela tivessem quebrado uma barreira invisível. “Corina... eu me lembro,” disse ele, com a voz trêmula. Ele explicou, ainda confuso, que havia sido capturado por Tenebris, que o enfeitiçou e o forçou a roubar a luz do céu.",
            "characters": ["Lívia", "Marildo", "Corina", "Caelus (After the Rescue)"],
            "items": ["The Orb"]
        },
        {
            "number": 10,
            "title": "Cena 10: A verdade sobre Tenebris e a decisão",
            "content": "Enquanto Caelus recuperava suas memórias, ele olhou para o chão com tristeza. “Fui eu quem roubou a luz para ela... Mas eu não queria. Ela usou minha dor e me controlou.” Corina colocou uma asa em seu ombro. “Você estava sob um feitiço, mas agora está livre. E juntos, podemos desfazer isso.” Nesse momento, os animais começaram a se aproximar, atentos. Entre eles, Tenebris, agora sem poderes, caminhava lentamente. “Você me salvou,” disse ela, olhando para Caelus. “Por quê?” Caelus hesitou, mas respondeu: “Porque todos merecem uma segunda chance. Até você.” Surpresa pela compaixão, Tenebris abaixou a cabeça.",
            "characters": ["Lívia", "Marildo", "Corina", "Caelus (After the Rescue)", "Tenebris", "Forest Animals"],
            "items": []
        },
        {
            "number": 11,
            "title": "Cena 11: Restaurando a luz ao céu",
            "content": "Livinha, Caelus e Corina trabalharam juntos para abrir o orbe que continha a luz roubada. Corina ergueu a lanterna mágica e a conectou ao orbe. Uma explosão de luz iluminou o vilarejo, enquanto as estrelas e a lua voltavam aos seus lugares no céu. Os animais ao redor comemoraram, e a floresta parecia respirar aliviada. A luz banhava cada folha e cada galho, e o brilho retornava às árvores mágicas. Caelus olhou para o céu, e lágrimas escorriam de seus olhos. “Finalmente... o céu está inteiro novamente.”",
            "characters": ["Lívia", "Corina", "Caelus (After the Rescue)", "Forest Animals"],
            "items": ["The Orb", "The Lantern"]
        },
        {
            "number": 12,
            "title": "Cena 12: A despedida de Tenebris",
            "content": "Enquanto o vilarejo celebrava, Tenebris ficou à margem, observando em silêncio. Livinha caminhou até ela. “Você pode começar de novo,” disse Livinha. Tenebris ergueu os olhos, surpresos. “Eu destruí tanto... não sei como.” “Com um passo de cada vez,” respondeu Corina, aproximando-se. “Vá. Encontre seu próprio caminho. A floresta é grande, e você terá sua chance de fazer algo bom.” Sem palavras, Tenebris se virou e desapareceu na sombra das árvores, deixando o vilarejo em paz.",
            "characters": ["Lívia", "Corina", "Tenebris"],
            "items": []
        },
        {
            "number": 13,
            "title": "Cena 13: O reconhecimento de Livinha",
            "content": "Quando a festa começou a acalmar, Corina foi até Livinha. “Você fez algo incrível hoje,” disse a Sentinela. “Não apenas trouxe a luz de volta, mas mostrou que a coragem e a bondade podem mudar qualquer coisa.” Marildo, ouvindo isso, levantou as orelhas. “Não se esqueça de que eu ajudei também!” Todos riram, e Livinha abraçou seu amigo. “Eu não teria feito isso sem você, Marildo,” disse ela.",
            "characters": ["Lívia", "Corina", "Marildo"],
            "items": []
        },
        {
            "number": 14,
            "title": "Cena 14: Livinha retorna para casa",
            "content": "Ao atravessar a porta mágica, Livinha encontrou seu pai e sua mãe esperando por ela, os rostos cheios de preocupação e alívio. “Estávamos tão preocupados!” disse sua mãe, abraçando-a forte. “Eu vivi uma grande aventura,” contou Livinha, com os olhos brilhando de alegria. Naquela noite, Livinha olhou para o céu estrelado da janela de seu quarto. As estrelas pareciam piscar para ela, como se agradecessem. Segurando as lembranças de sua aventura, Livinha sussurrou: “Até breve, amigos,” e deixou os sonhos de luz e coragem embalarem seu sono.",
            "characters": ["Lívia", "Lívia's Parents"],
            "items": ["The Magic Door"]
        }
    ]
}
type(story)

In [7]:
style_description = """Whimsical, colorful and children's fantasy storybook drawings in the artistic style of 'The Gruffalo' and 'The Snail and the Whale'"""

In [ ]:
characters_list_dict = [
    {
        "name": "Lívia",
        "appearance": "Lívia is a 3-year-old girl with loose shoulder-length golden-blonde straight hair that frames her rosy cheeks. Her large, dark green eyes shine with curiosity. Her skin is fair with a light blush, and she stands about half the height of an adult. Lívia wears comfortable fit medium blue denim overalls over a plain light pastel pink short-sleeve T-shirt. Her shoes are small, unadorned brown leather shoes suitable for exploring. The overalls have simple straps and a front pocket.",
        "personality_visualized": "Lívia’s posture is confident and playful, often shown in motion—reaching toward a butterfly, glancing over her shoulder with excitement, or walking through the glowing forest. Her expressions range from awe to determination, emphasizing her bravery and kind heart."
    },
    {
        "name": "Marildo",
        "appearance": "Marildo is a white rabbit with soft, fluffy fur and large, floppy ears. His eyes are round and expressive, colored bright blue. He is about the size of a small cat. Marildo wears a plain sky blue hoodie that's slightly oversized. The hoodie has a front pocket, and the sleeves are rolled up, showing his small white paws.",
        "personality_visualized": "Marildo’s wide grin and twitching nose convey his quick thinking and humor. He’s often shown hopping energetically or striking a triumphant pose after solving a puzzle or making a clever escape."
    },
    {
        "name": "Corina",
        "appearance": "Corina is a golden-brown falcon with softly shimmering feathers. She has large, warm amber eyes. The tips of her wings and tail feathers are pure white, resembling moonlight. She is about twice the size of Marildo.",
        "personality_visualized": "Corina has an aura of calm and wisdom. Her movements are graceful, and she is often depicted perched on a low branch, her wings partially open as if ready to protect or guide. She has a gentle but authoritative expression, showing her role as a Sentinel."
    },
    {
        "name": "Caelus (Before the Rescue)",
        "appearance": "Before his rescue, Caelus is a large falcon with deep black feathers that absorb light. His eyes are dull gold, appearing dim and shadowed. He is larger than Corina, making him an imposing figure.",
        "personality_visualized": "Caelus is hunched over, with a defensive, but almost menacing posture. His movements are slow and heavy, as if weighed down by the spell keeping him captive. He rarely looks anyone in the eye, projecting both sadness and a restrained power."
    },
    {
        "name": "Caelus (After the Rescue)",
        "appearance": "After being freed, Caelus is a large black falcon with subtle golden markings on his feathers. His eyes are bright gold, shining with clarity. He remains larger than Corina, appearing regal and proud.",
        "personality_visualized": "Caelus stands tall and confident, his wings spread wide as if ready to soar. His expressions are soft, showing gratitude and a desire to make amends. He is often shown close to Corina, symbolizing their reunion."
    },
    {
        "name": "Tenebris",
        "appearance": "Tenebris is a tall woman with pale skin that reflects softly like moonlight. Her long hair is deep black with silver streaks resembling starlight. She has piercing violet eyes that glow faintly. She wears a flowing black robe with simple silver star patterns along the hem and sleeves.",
        "personality_visualized": "Tenebris is depicted with an air of confidence and menace, her posture upright and commanding. However, in moments of vulnerability, her expression softens, revealing the sadness behind her destructive actions."
    }
]
type(characters_list_dict)

In [ ]:
items_list_dict = [
    {
        "name": "The Orb",
        "appearance": "A floating sphere about the size of a soccer ball. It looks like clear glass and emits a soft glow. Inside, gentle swirls of silver and pale gold light move slowly, resembling stars and moonlight. The orb's surface is smooth and slightly luminous."
    },
    {
        "name": "The Lantern",
        "appearance": "A small, hand-held lantern made of polished silver metal with gold accents. It has simple carvings of stars and crescent moons on its sides. Inside, a softly glowing light flickers between warm golden-yellow and cool silver-blue. It has a curved handle on top for carrying."
    },
    {
        "name": "The Map",
        "appearance": "An aged parchment scroll, about the size of an open magazine when unrolled. It has hand-drawn lines and simple symbols that glow softly in light gold. Small five-pointed stars and crescent moons mark important locations. The map is rolled up and tied with a thin golden ribbon."
    },
    {
        "name": "The Key",
        "appearance": "A simple gold key about three inches long. It has a star-shaped head with five points and a small crescent moon at the base of the handle. The key shines brightly, making it stand out."
    },
    {
        "name": "The Magic Door",
        "appearance": "A small, rounded wooden door that magically hangs in the air. It is made of dark brown wood with a smooth texture. A simple golden star is embedded in the wood. Moss and tiny."
    }
]
type(items_list_dict)

In [ ]:
synopsis = """When the night sky mysteriously goes dark, a young girl named Livinha finds a golden key that leads her through a magical portal into a hidden forest. There she meets Corina, a wise falcon, and Marildo, a brave rabbit, and learns that a villain named Tenebris has stolen all the starlight and moonlight, cursing the magical Sentinel family who protect the forest. With courage and compassion, Livinha and her friends free Corina's captive brother Caelus, restore the light to the sky, and show that even those who seem lost in darkness deserve a second chance."""
len(synopsis)

# Image Generation Function

In [11]:
from openai import AzureOpenAI, OpenAI
import os
import json

# TODO: Test with OpenAI directly. AzureOpenAI rewrites all prompts, compromising consistency in image generation
def generate_illustration(user_prompt: str, aspect_ratio: str = "wide", provider: str = "azure") -> str:
    """
    Generate children's books illustrations using Azure OpenAI's DALL-E model.

    Args:
        user_prompt (str): The prompt to generate the artwork.
        n (int, optional): The number of images to generate. Defaults to 1.
        aspect_ratio (str, optional): Determines the dimensions of the generated image. Defaults to "wide".
        provider (str, optional): The AI provider to use. Defaults to "azure".

    Returns:
        str: The URL of the generated image.
    """
    if provider == "azure":
        client = AzureOpenAI(
            api_version="2024-02-01",  
            api_key=os.environ["AZURE_OPENAI_API_KEY"],  
            azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
        )
    else:
        client = OpenAI(
            api_key=os.environ["OPENAI_API_KEY"]
        )

    image_sizes = {
    "square": "1024x1024",
    "wide": "1792x1024",
    "vertical": "1024x1792"
    }

    size = image_sizes[aspect_ratio]
    
    # print("Generating Image")
    new_image = client.images.generate(
        model="dall-e-3",  # the name of your DALL-E 3 deployment
        prompt=user_prompt,
        n=1,
        size=size,
        style="vivid",
        quality="hd"
    )

    json_response = json.loads(new_image.model_dump_json())
    image_url = json_response["data"][0]["url"]  # extract image URL from response
    print(image_url)
    return image_url

In [12]:
prompt_template = """# SYNOPSIS
{synopsis}

# STYLE
{style_description}

# SCENE
{scene}

# CHARACTERS
{characters}

# ITEMS
{items}
"""

In [13]:
def format_pre_prompt(scene, story_title, characters_list_dict, items_list_dict, prompt_template, synopsis, style_description):
    scene_title = scene['title']
    scene_content = scene['content']

    characters = ""
    for character_name in scene['characters']:
        character = next((char for char in characters_list_dict if char['name'] == character_name), None)
        if character:
            character_appearance = character['appearance']
            characters += f"""\n## {character_name}\n{character_appearance}\n\n"""

    items = ""
    for item_name in scene['items']:
        item = next((itm for itm in items_list_dict if itm['name'] == item_name), None)
        if item:
            item_appearance = item['appearance']
            items += f"""\n## {item_name}\n{item_appearance}\n\n"""
    
    pre_prompt = prompt_template.format(
        book_title=story_title,
        synopsis=synopsis,
        style_description=style_description,
        scene=scene_title + '\n' + scene_content,
        characters=characters,
        items=items
    )

    return {
        "number": scene['number'],
        "scene_title": scene_title,
        "formatted_prompt_str": pre_prompt
    }

In [14]:
formatted_prompts = []

for scene in story["scenes"]:
    formatted_prompt = format_pre_prompt(
        scene=scene,
        story_title="Livinha e a Chave Encantada: Em busca das luzes roubadas",
        characters_list_dict=characters_list_dict,
        items_list_dict=items_list_dict,
        prompt_template=prompt_template,
        synopsis=synopsis,
        style_description=style_description
    )
    formatted_prompts.append(formatted_prompt)

In [15]:
def generate_improved_prompt(pre_prompt: str, max_length: int = 1000) -> str:
    tools = [calculate_text_length]
    custom_prompt = "# Persona\nYou are an expert at using AI image generation for creating children's books illustrations. Your job is to optimize this pre-prompt for generating consistent and high-quality illustrations."
    agent = MyAgent(tools=tools, custom_prompt=custom_prompt)
    
    instructions = f"""# Task
In {max_length} characters, turn the pre-prompt into a detailed visual description of a static scene for a book illustration.
Make sure to specify sizes, colors, directions, textures and the like.
Do it such that if I use the same improved prompt multiples times with the same model and settings, the resulting images will be extremely similar, like 90% the same.
Below, is the pre-prompt for the scene we're about to illustrate, delimited by triple backticks. Please enhance it to ensure the generated illustration is as close to the story as possible.
Your response MUST be {max_length} characters or less.

# IMPORTANT
Use 'calculate_text_length' to ensure your response is within the character limit before submitting a final response.
If it's longer than {max_length} characters, revise it to fit the limit.
Remove repetitive or unnecessary details and shorten words and sentences whenever possivle, while keeping the essence of the scene intact.
Feel free to replace expressions with shorter synonyms, remove redundant words, or rephrase sentences to be more concise.

# Pre-Prompt
{pre_prompt}"""
    
    improved_prompt = agent.myinvoke(instructions)
    
    print(f"\n{improved_prompt}\n")
    return improved_prompt

In [16]:
def save_image_from_url(image_url: str, save_path: str):
    """
    Downloads an image from a URL and saves it locally.

    Args:
        image_url (str): The URL of the image to download.
        save_path (str): The local path where the image will be saved.
    """
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image saved to {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

In [17]:
def process_story_scenes(story: dict, characters_list_dict: list, items_list_dict: list, synopsis: str, style_description: str, prompt_template: str, prompt_max_length: int = 4000, scene_numbers: list = None, display_images: bool = True) -> list:

    # Create images and prompts directories if they don't exist
    os.makedirs("images", exist_ok=True)
    os.makedirs("prompts", exist_ok=True)

    generated_scenes = []

    for scene in story['scenes']:
        if scene_numbers and scene['number'] not in scene_numbers:
            continue

        # Format the pre prompt
        pre_prompt = format_pre_prompt(
            scene=scene,
            story_title=story['title'],
            characters_list_dict=characters_list_dict,
            items_list_dict=items_list_dict,
            prompt_template=prompt_template,
            synopsis=synopsis,
            style_description=style_description
        )
        
        improved_prompt_dict = generate_improved_prompt(pre_prompt, max_length=prompt_max_length)
        improved_prompt = improved_prompt_dict['output']

        # Save the improved prompt
        timestamp = time.strftime("%Y%m%d%H%M%S")
        prompt_file_path = f"prompts/{timestamp}_scene_{scene['number']}.txt"
        with open(prompt_file_path, 'w') as file:
            file.write(improved_prompt + '\n')

        # # Generate and save illustration
        # image_file_path = f"images/scene_{scene['number']}_{timestamp}.png"
        # try:
        #     cli_print(f"Generating illustration for scene: {scene['title']}")
        #     image_url = generate_illustration(user_prompt=improved_prompt, provider="azure")
        #     save_image_from_url(image_url, save_path=image_file_path)
        #     generated_scenes.append({"scene_number": scene['number'], "file_path": image_file_path})
        # except Exception as e:
        #     cli_print(f"Failed to generate or save illustration for scene '{scene['title']}'. Error: {e}")
        #     continue

        # # Optionally display the illustration at 20% original size
        # if display_images:
        #     display(Image(filename=image_file_path, width="30%"))

    return generated_scenes

In [ ]:
generated_scenes = process_story_scenes(story, characters_list_dict, items_list_dict, synopsis, style_description, prompt_template, prompt_max_length = 1500, display_images = True)